In [44]:
# 4DfCF_
import pandas as pd
from monai.transforms import LoadImage
import torch
import os
import time
from multiprocessing import Process, Queue
import shutil

In [45]:
def read_data(filename,load_root,save_root,subj_name,count,queue=None,scaling_method=None, fill_zeroback=False):
    print("processing: " + filename, flush=True)
    path = os.path.join(load_root, filename)
    try:
        # load each nifti file
        data, meta = LoadImage()(path)
    except:
        return None
    
    #change this line according to your file names
    save_dir = os.path.join(save_root,subj_name)
    isExist = os.path.exists(save_dir)
    if not isExist:
        os.makedirs(save_dir)
    
    # change this line according to your dataset
    data = data[:, 14:-7, :, :]
    # width, height, depth, time
    # Inspect the fMRI file first using your visualization tool. 
    # Limit the ranges of width, height, and depth to be under 96. Crop the background, not the brain regions. 
    # Each dimension of fMRI registered to MNI space (2mm) is expected to be around 100.
    # You can do this when you load each volume at the Dataset class, including padding backgrounds to fill dimensions under 96.
   
    background = data==0
    
    if scaling_method == 'z-norm':
        global_mean = data[~background].mean()
        global_std = data[~background].std()
        data_temp = (data - global_mean) / global_std
    elif scaling_method == 'minmax':
        data_temp = (data - data[~background].min()) / (data[~background].max() - data[~background].min())

    data_global = torch.empty(data.shape)
    data_global[background] = data_temp[~background].min() if not fill_zeroback else 0 
    # data_temp[~background].min() is expected to be 0 for scaling_method == 'minmax', and minimum z-value for scaling_method == 'z-norm'
    data_global[~background] = data_temp[~background]

    # save volumes one-by-one in fp16 format.
    data_global = data_global.type(torch.float16)
    data_global_split = torch.split(data_global, 1, 3)
    for i, TR in enumerate(data_global_split):
        torch.save(TR.clone(), os.path.join(save_dir,"frame_"+str(i)+".pt"))


 


In [48]:
def main(root_folder, dataset_name):
    subfolders = [f.path for f in os.scandir(root_folder) if f.is_dir()]
    
    for subfolder in subfolders:
        folder_name = os.path.basename(subfolder)
        # change two lines below according to your dataset
        # dataset_name = 'NYU'
        load_root = f"{root_folder}/{dataset_name}" # This folder should have fMRI files in nifti format with subject names. Ex) sub-01.nii.gz 
        save_root = f'../../../../data/preprocessed_data/{dataset_name}/7.{dataset_name}_MNI_to_TRs_minmax'
        scaling_method = 'z-norm' # choose either 'z-norm'(default) or 'minmax'.
    
        # make result folders
        filenames = os.listdir(load_root)
        
        filenames = [filename for filename in filenames if not filename.endswith(".sh")]
        # print(filenames)
        
        os.makedirs(os.path.join(save_root,'img'), exist_ok = True)
        os.makedirs(os.path.join(save_root,'metadata'), exist_ok = True) # locate your metadata file at this folder 
        save_root = os.path.join(save_root,'img')
        
        finished_samples = os.listdir(save_root)
        queue = Queue() 
        count = 0
        for filename in sorted(filenames):
            # print(filename)
            subj_name = []
            if filename.startswith("Peking"):
                subj_name = filename[9:-7]
            elif filename.startswith("Neuro"):
                subj_name = filename[11:-7]
            elif filename.startswith("KKI"):
                subj_name = filename[18:-7]
                subj_name = "sfnwmrda" + subj_name
            elif filename.startswith("Pitts"):
                subj_name = filename[11:-7]
            elif filename.startswith("NYU"):
                subj_name = filename[4:-7]
            elif filename.startswith("OHSU_niak"):
                subj_name = filename[19:-7]
                subj_name = "sfnwmrda" + subj_name
            elif filename.startswith("OHSU_sfn"):
                subj_name = filename[5:-7]
            elif filename.startswith("WashU"):
                subj_name = filename[6:-7]
                
            # extract subject name from nifti file. [:-7] rules out '.nii.gz'
            # we recommend you use subj_name that aligns with the subject key in a metadata file.
    
            expected_seq_length = 1000 # Specify the expected sequence length of fMRI for the case your preprocessing stopped unexpectedly and you try to resume the preprocessing.
            
            # change the line below according to your folder structure
            if (subj_name not in finished_samples) or (len(os.listdir(os.path.join(save_root,subj_name))) < expected_seq_length): # preprocess if the subject folder does not exist, or the number of pth files is lower than expected sequence length. 
                try:
                    count+=1
                    p = Process(target=read_data, args=(filename,load_root,save_root,subj_name,count,queue,scaling_method))
                    p.start()
                    if count % 32 == 0: # requires more than 32 cpu cores for parallel processing
                        p.join()
                except Exception:
                    print('encountered problem with'+filename)
                    print(Exception)
    
        # csv_file = f"{root_folder}/{dataset_name+'_phenotypic.csv'}"
        # shutil.copy(csv_file, os.path.join(save_root, 'metadata'))
  

In [50]:
print(os.path.exists("../../../../../../FMRI_ADHD_Classification/Extracted_FMRI_Data/Total_Data_Pheno.csv"))

True


In [ ]:
shutil.copy("../../../../../../FMRI_ADHD_Classification/Extracted_FMRI_Data/Total_Data_Pheno.csv", )

In [51]:
# root_folder = '../../../../data/FMRI_Data'
dataset_name = 'ADHD_all'
csv_file = "../../../../../../FMRI_ADHD_Classification/Extracted_FMRI_Data/Total_Data_Pheno.csv"
save_root = f'../../../../data/preprocessed_data/{dataset_name}/7.{dataset_name}_MNI_to_TRs_minmax'
shutil.copy(csv_file, os.path.join(save_root, 'metadata'))
# print(os.path.exists(os.path.join(save_root, 'metadata')))

'../../../../data/preprocessed_data/ADHD_all/7.ADHD_all_MNI_to_TRs_minmax/metadata/Total_Data_Pheno.csv'

In [52]:
old_csv = '../../../../data/preprocessed_data/ADHD_all/7.ADHD_all_MNI_to_TRs_minmax/metadata/Total_Data_Pheno.csv'
new_csv = '../../../../data/preprocessed_data/ADHD_all/7.ADHD_all_MNI_to_TRs_minmax/metadata/ADHD_all_phenotypic.csv'
os.rename(old_csv, new_csv)

In [53]:
import pandas as pd
csv_file = pd.read_csv('../../../../data/preprocessed_data/ADHD_all/7.ADHD_all_MNI_to_TRs_minmax/metadata/ADHD_all_phenotypic.csv')
csv_file

,Name,DX,Loc
0,Peking_3_sfnwmrda1662160_session_1_rest_1.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
1,Peking_3_sfnwmrda3624598_session_1_rest_1.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
2,Peking_3_sfnwmrda2107404_session_1_rest_1.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
3,Peking_3_sfnwmrda4048810_session_1_rest_1.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
4,Peking_3_sfnwmrda3291029_session_1_rest_1.nii.gz,1,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
...,...,...,...
842,WashU_sfnwmrda0015049_session_1_rest_1.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
843,WashU_sfnwmrda0015061_session_1_rest_3.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
844,WashU_sfnwmrda0015054_session_1_rest_1.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...
845,WashU_sfnwmrda0015058_session_1_rest_2.nii.gz,0,/home/researchsrv1/chris/demo/codes/FMRI_ADHD_...


In [54]:
csv_file.rename(columns={'Name': 'ScanDir ID'}, inplace=True)
csv_file.drop(columns=['Loc'], inplace=True)
csv_file

,ScanDir ID,DX
0,Peking_3_sfnwmrda1662160_session_1_rest_1.nii.gz,0
1,Peking_3_sfnwmrda3624598_session_1_rest_1.nii.gz,0
2,Peking_3_sfnwmrda2107404_session_1_rest_1.nii.gz,0
3,Peking_3_sfnwmrda4048810_session_1_rest_1.nii.gz,0
4,Peking_3_sfnwmrda3291029_session_1_rest_1.nii.gz,1
...,...,...
842,WashU_sfnwmrda0015049_session_1_rest_1.nii.gz,0
843,WashU_sfnwmrda0015061_session_1_rest_3.nii.gz,0
844,WashU_sfnwmrda0015054_session_1_rest_1.nii.gz,0
845,WashU_sfnwmrda0015058_session_1_rest_2.nii.gz,0


In [60]:
import pandas as pd

# 查找名为 'vava' 列中包含 'run' 字符串的行数
count = csv_file[csv_file['ScanDir ID'].str.contains('OHSU_niak', na=False)].shape[0]

# 打印结果
print("名为 'vava' 列中包含 'run' 字符串的行数为:", count)


名为 'vava' 列中包含 'run' 字符串的行数为: 79


In [61]:
# 查找名为 'vava' 列中包含 'run' 字符串的行索引
indexes_to_drop = csv_file[csv_file['ScanDir ID'].str.contains('OHSU_niak', na=False)].index

# 删除满足条件的行
csv_file.drop(indexes_to_drop, inplace=True)

# 打印结果
print("已删除满足条件的行，剩余行数为:", len(csv_file))

已删除满足条件的行，剩余行数为: 768


In [62]:
csv_file

,ScanDir ID,DX
0,Peking_3_sfnwmrda1662160_session_1_rest_1.nii.gz,0
1,Peking_3_sfnwmrda3624598_session_1_rest_1.nii.gz,0
2,Peking_3_sfnwmrda2107404_session_1_rest_1.nii.gz,0
3,Peking_3_sfnwmrda4048810_session_1_rest_1.nii.gz,0
4,Peking_3_sfnwmrda3291029_session_1_rest_1.nii.gz,1
...,...,...
842,WashU_sfnwmrda0015049_session_1_rest_1.nii.gz,0
843,WashU_sfnwmrda0015061_session_1_rest_3.nii.gz,0
844,WashU_sfnwmrda0015054_session_1_rest_1.nii.gz,0
845,WashU_sfnwmrda0015058_session_1_rest_2.nii.gz,0


In [64]:
import os

# 文件夹路径
folder_path = '../../../../data/preprocessed_data/ADHD_all/7.ADHD_all_MNI_to_TRs_minmax/img'

# 遍历文件夹中的所有文件夹
for folder_name in os.listdir(folder_path):
    # 构建文件夹的完整路径
    folder_full_path = os.path.join(folder_path, folder_name)
    
    # 检查文件夹名称是否以 'abc' 开头
    if folder_name.startswith('OHSU_niak'):
        # 如果是，则删除文件夹
        os.rmdir(folder_full_path)


In [68]:
import os

# 存储已经遇到的数字
encountered_numbers = set()

# 遍历文件夹中的所有文件夹
for folder_name in os.listdir(folder_path):
    # 检查文件夹名称是否满足指定格式
    if folder_name.startswith('sfnwmrda'):
        # 提取七位数字部分
        number_part = folder_name[8: 15]
        
        # 检查是否遇到相同的七位数字
        if number_part in encountered_numbers:
            # 如果是，删除文件夹
            shutil.rmtree(os.path.join(folder_path, folder_name))
            deleted_count += 1
        else:
            # 如果不是，将数字添加到已经遇到的数字集合中
            encountered_numbers.add(number_part)

# 输出删除的文件夹数量
print("Deleted {} folders.".format(deleted_count))

Deleted 79 folders.


In [71]:
# 遍历文件夹中的所有文件夹
for folder_name in os.listdir(folder_path):
    # 检查文件夹名称是否满足指定格式
    if folder_name.startswith('sfnwmrda'):
        # 分割文件夹名称，以 '_session' 为分隔符
        parts = folder_name.split('_session')
        # 如果存在 '_session'，则修改文件夹名称
        if len(parts) > 1:
            new_folder_name = parts[0] + '_session'
            # 构建原始文件夹路径和新文件夹路径
            original_folder_path = os.path.join(folder_path, folder_name)
            new_folder_path = os.path.join(folder_path, new_folder_name)
            # 重命名文件夹
            os.rename(original_folder_path, new_folder_path)
            print(f"Renamed folder from '{folder_name}' to '{new_folder_name}'")

Renamed folder from 'sfnwmrda1643780_session_1_rest_1' to 'sfnwmrda1643780_session'
Renamed folder from 'sfnwmrda2352986_session_1_rest_1' to 'sfnwmrda2352986_session'
Renamed folder from 'sfnwmrda1481430_session_1_run1' to 'sfnwmrda1481430_session'
Renamed folder from 'sfnwmrda1418396_session_1_rest_2' to 'sfnwmrda1418396_session'
Renamed folder from 'sfnwmrda2292940_session_1_rest_1' to 'sfnwmrda2292940_session'
Renamed folder from 'sfnwmrda6453038_session_1_run1' to 'sfnwmrda6453038_session'
Renamed folder from 'sfnwmrda1094669_session_1_rest_1' to 'sfnwmrda1094669_session'
Renamed folder from 'sfnwmrda3679455_session_1_rest_2' to 'sfnwmrda3679455_session'
Renamed folder from 'sfnwmrda1282248_session_1_rest_1' to 'sfnwmrda1282248_session'
Renamed folder from 'sfnwmrda3967265_session_1_rest_1' to 'sfnwmrda3967265_session'
Renamed folder from 'sfnwmrda2427434_session_1_run2' to 'sfnwmrda2427434_session'
Renamed folder from 'sfnwmrda0010084_session_1_rest_1' to 'sfnwmrda0010084_session

In [69]:
def modify(value):
    if value.startswith("Peking"):
        value = value[17:-24]
    elif value.startswith("Neuro"):
        value = value[19:-24]
    elif value.startswith("KKI"):
        value = value[18:-22]
    elif value.startswith("Pitts"):
        value = value[19:-24]
    elif value.startswith("NYU"):
        value = value[12:-24]
    elif value.startswith("OHSU_sfn"):
        value = value[13:-24]
    elif value.startswith("OHSU_niak"):
        value = value[19:-22]    
    elif value.startswith("WashU"):
        value = value[14:-24]
    return value.upper()

csv_file['ScanDir ID'] = csv_file['ScanDir ID'].apply(modify)
csv_file

,ScanDir ID,DX
0,1662160,0
1,3624598,0
2,2107404,0
3,4048810,0
4,3291029,1
...,...,...
842,0015049,0
843,0015061,0
844,0015054,0
845,0015058,0


In [70]:
csv_file.to_csv('../../../../data/preprocessed_data/ADHD_all/7.ADHD_all_MNI_to_TRs_minmax/metadata/ADHD_all_phenotypic.csv')

In [71]:
import os

# 文件夹路径
folder_path = '../../../../data/FMRI_data'

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    # 构建文件的完整路径
    file_path = os.path.join(folder_path, filename)
    
    # 如果文件名的第一个字为数字
    if filename[0].isdigit():
        # 构建新的文件名，加上 'asdfg' 前缀
        new_filename = 'asdfg' + filename
        # 重命名文件
        os.rename(file_path, os.path.join(folder_path, new_filename))
        
    # 如果文件名以 'fmri' 开头
    if filename.startswith('fmri'):
        # 构建新的文件名，将 'fmri' 改为 'asdfg'
        new_filename = 'asdfg' + filename[4:]
        # 重命名文件
        os.rename(file_path, os.path.join(folder_path, new_filename))


In [20]:
import os

# 文件路径
file_path = '../../../../data'

# 检查文件是否存在
if os.path.exists(file_path):
    print("文件存在")
else:
    print("文件不存在")
subfolders = [f.path for f in os.scandir(file_path) if f.is_dir()]
print(subfolders)

文件存在
['../../../../data/preprocessed_data', '../../../../data/FMRI_Data', '../../../../data/.ipynb_checkpoints']


In [49]:
if __name__=='__main__':
    root_folder = '../../../../data/FMRI_data'
    dataset_name = 'ADHD_all'
    
    start_time = time.time()
    main(root_folder, dataset_name)
    end_time = time.time()
        
    print('\nTotal', round((end_time - start_time) / 60), 'minutes elapsed.') 

processing: KKI_niak_1_fmri_X_1018959_session_1_run1.nii.gz


/home/researchsrv1/anaconda3/envs/chris_adhd_swift_py39/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.array LoadImage.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


processing: KKI_niak_1_fmri_X_1019436_session_1_run1.nii.gz


/home/researchsrv1/anaconda3/envs/chris_adhd_swift_py39/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.array LoadImage.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


processing: KKI_niak_1_fmri_X_1043241_session_1_run1.nii.gz

processing: KKI_niak_1_fmri_X_1266183_session_1_run1.nii.gz

...

/home/researchsrv1/anaconda3/envs/chris_adhd_swift_py39/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.array LoadImage.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


/home/researchsrv1/anaconda3/envs/chris_adhd_swift_py39/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.array LoadImage.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)



Total 34 minutes elapsed.
processing: WashU_sfnwmrda0015061_session_1_rest_3.nii.gzprocessing: WashU_sfnwmrda0015060_session_1_rest_1.nii.gz

/home/researchsrv1/anaconda3/envs/chris_adhd_swift_py39/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.array LoadImage.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


processing: WashU_sfnwmrda0015059_session_1_rest_2.nii.gz